### Learn Azure in Hong Kong | Multimodel Multimodal and Multiagent with Azure

Cantonese series talking about everything related to AI on Azure.  
廣東話系列講述與 Azure AI 相關的一切。  
广东话讲述与 Azure AI 相关的一切。

Addressing real-world scenarios with AI models requires a diverse approach. Each model may offer unique capabilities, and numerous types of source files, such as text, image, and video, need to be considered.

In this session, we will demonstrate practical applications of multimodal and multiagent sample using Azure AI services. Join us for this Microsoft Reactor session to learn how to manage these varied scenarios and benefit from insights on actual case experiences.

處理實際場景中的 AI 模型需要多元化的方法。每個模型可能提供獨特的功能，且需要考慮多種類型的源文件，如文本、圖像和影片。

在本次分享中，我們將演示使用 Azure AI 服務的多模式和多代理範例的實際應用。加入我們的 Microsoft Reactor 會議，了解如何管理這些不同的場景，並從對實際案例體驗的見解中受益。

解决现实场景中的 AI 模型问题需要多样化的方法。每个模型可能提供独特的能力，并且需要考虑多种类型的源文件，如文本、图像和视频。

在本次分享中，我们将展示如何使用 Azure AI 服务进行多模态和多代理的实际应用示例。加入我们，了解如何管理这些多样化的场景，并从实际案例经验中获益。

Event time: 28 Feb 2025 19:30-20:30  
Event Type: Microsoft Reactor online live  
Event URL: https://developer.microsoft.com/en-us/reactor/events/25041/


In [ ]:
# Provide the information of database connection
db_conn = {
    'host': 'localhost',  # Database server hostname or IP address
    'port': '3306',       # Database connection port number
    'user': 'root',       # Database username
    'password': '123456', # Database password
    'database': 'shipment' # Name of database to connect to
}

In [ ]:
# Provide the database schema for Text2SQL model
db_schema = """
CREATE TABLE shipment (
  Sales_Document int DEFAULT NULL,
  Delivery bigint DEFAULT NULL,
  Customer_Name text COLLATE utf8mb3_unicode_ci,
  Site_Name text COLLATE utf8mb3_unicode_ci,
  Delivery_block text COLLATE utf8mb3_unicode_ci,
  Status int DEFAULT NULL,
  Truck text COLLATE utf8mb3_unicode_ci,
  Material_Description text COLLATE utf8mb3_unicode_ci,
  Shipping_PointReceiving_Pt text COLLATE utf8mb3_unicode_ci,
  Delivery_quantity double DEFAULT NULL,
  Date_arrive_Plant text COLLATE utf8mb3_unicode_ci,
  Time_arrive_Plant text COLLATE utf8mb3_unicode_ci,
  Date_of_Loading text COLLATE utf8mb3_unicode_ci,
  Start_time_of_loading text COLLATE utf8mb3_unicode_ci,
  Date_leave_plant text COLLATE utf8mb3_unicode_ci,
  Time_leave_plant text COLLATE utf8mb3_unicode_ci,
  Date_arrive_site text COLLATE utf8mb3_unicode_ci,
  Time_arrive_site text COLLATE utf8mb3_unicode_ci,
  Travelling_Time int DEFAULT NULL,
  Date_start_unloading text COLLATE utf8mb3_unicode_ci,
  Time_start_unloading text COLLATE utf8mb3_unicode_ci,
  Date_leave_site text COLLATE utf8mb3_unicode_ci,
  Time_leave_site text COLLATE utf8mb3_unicode_ci,
  Wating_Time int DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb3 COLLATE=utf8mb3_unicode_ci;
"""

In [ ]:
question = "Which products were delivered the most?"

In [ ]:
# The prompt for TEXT2SQL model
text2sql_prompt = f"""### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Database Schema
{db_schema}

### Instructions
As an AI chatbot specialized in SQL queries for MySQL databases, your task is to generate a syntactically correct MySQL query to retrieve information from the provided database schema. Follow these guidelines:
- Utilize your expertise in MySQL to craft efficient queries that return relevant data.
- Use the schema provided to identify the appropriate tables and columns for your query.
- If the question involves conditions such as "today" or "current", utilize the GETDATE() function to retrieve the current date and time.
- Employ joins between tables if necessary to gather relevant information.
- Query for at most [top_k] results using the OFFSET FETCH clause for performance optimization. You can specify the desired value for [top_k] when calling the function.
- Select only the columns necessary to answer the question. Enclose each column name in square brackets ([]) to denote them as delimited identifiers.
- Avoid querying for all columns from a table to enhance efficiency.
- If the question cannot be answered with the available database schema, return 'I do not know'.
- ***When ordering results, handle NULL values appropriately and DO NOT using the 'NULLS LAST' syntax.***

**Missing Schema Handling:**
If the database schema information is missing, please provide it for query generation. Otherwise, this process will return 'Missing database schema'.

### Answer
Given the database schema, here is the SQL query that [QUESTION]{question}[/QUESTION]
[SQL]"""

In [ ]:
import requests
import json

# Set the URL for the local Ollama server (adjust the port if needed)
url = "http://localhost:11434/run"

# Prepare the payload with the model name and prompt
payload = {
    "model": "defog_sqlcoder-7b-2",
    "prompt": text2sql_prompt,  # using the variable from cell index 4
    "max_tokens": 1024          # adjust token limit if necessary
}

headers = {
    "Content-Type": "application/json"
}

# Send a POST request to the Ollama endpoint
response = requests.post(url, headers=headers, json=payload)

if response.status_code == 200:
    result = response.json()
    # Save the output response in string format into a variable
    response_str = json.dumps(result)
    print("Generated SQL Query:")
    print(result)
else:
    print("Request failed with status code:", response.status_code)
    print(response.text)

In [ ]:
import pymysql  # Import library for MySQL database interaction
import json    # Import library for JSON data processing

def main(json_data, sql_command):
    """
    Connect to MySQL database and execute the given SQL command.

    Parameters:
    json_data (str): JSON string containing database connection information.
    sql_command (str): SQL query command to execute.

    Returns:
    dict: Dictionary containing query results or error message.
    """
    try:
        # Parse JSON string into Python dictionary
        config = json.loads(json_data)
        
        # Establish database connection using configuration info
        connection = pymysql.connect(
            host=config['host'],        # Database server hostname or IP address
            port=config['port'],        # Database connection port number
            user=config['user'],        # Database username
            password=config['password'], # Database password
            database=config['database_name'] # Name of database to connect to
        )
        
        try:
            with connection.cursor() as cursor:  # Get database cursor using context manager
                # Execute SQL command
                cursor.execute(sql_command)
                
                # Get all query results
                result = cursor.fetchall()
                
                # Convert results to JSON string format
                #result_str = json.dumps(result, ensure_ascii=False)
                
                # Return dictionary containing query results
                return {
                    "result": str(result)
                }
        
        finally:
            # Ensure database connection is closed to free resources
            connection.close()
    
    except Exception as e:
        # Catch and return any exception information
        return {
            "result": f"Error: {str(e)}"
        }